<a href="https://colab.research.google.com/github/dropcreations/MKVToolNix-in-Google-Colab/blob/main/MKVToolNix-in-Google-Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __Mount Google Drive__

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/dropcreations/MKVToolNix-in-Google-Colab/main/Google-Drive-Logo.svg' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3><b>Mount Google Drive</b></h3></center><br>

from google.colab import drive

Mode = "Mount" #@param ["Mount", "Unmount"]

drive.mount._DEBUG = False

if Mode == "Mount":
    drive.mount('/content/drive', force_remount=True)

elif Mode == "Unmount":
    try:
        drive.flush_and_unmount()
    except ValueError:
        pass
    get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

# __MKVToolNix__

* Run below cell to **install** MKVToolNix.
* You can use any version, to do that enter the version in `Version`.
* If you want to install `version 70.0.0`, just type `70` in `Version`.

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/dropcreations/MKVToolNix-in-Google-Colab/main/MKVToolNix-Logo.png' height="50" alt="mkvtoolnix-logo"/></center>
#@markdown <center><h3><b>Install MKVToolNix</b></h3></center><br>

import json
import requests
from IPython.display import clear_output

Version = "latest-release" #@param ["latest-release"] {allow-input: true}

response = requests.get("https://mkvtoolnix.download/releases.json")
release_info = json.loads(response.text)
latest_version = release_info["mkvtoolnix-releases"]["latest-source"].get("version")

releases = []

for i in range(len(release_info["mkvtoolnix-releases"].get("releases"))):
    ver = release_info["mkvtoolnix-releases"]["releases"][i].get("version")
    releases.append(ver)

if Version == "latest-release":
    download_link = f'https://mkvtoolnix.download/appimage/MKVToolNix_GUI-{latest_version}-x86_64.AppImage'
else:
    for match in sorted(releases):
        if Version in match:
            Version = match
    download_link = f'https://mkvtoolnix.download/appimage/MKVToolNix_GUI-{Version}-x86_64.AppImage'

!rm -f '/usr/local/bin/mkvpropedit'
!rm -f '/usr/local/bin/mkvmerge'
!rm -f '/usr/local/bin/mkvextract'
!rm -f '/usr/local/bin/mkvinfo'

!sudo curl -L {download_link} -o /usr/local/bin/MKVToolNix_GUI.AppImage
!sudo chmod u+rx /usr/local/bin/MKVToolNix_GUI.AppImage
!sudo ln -s /usr/local/bin/MKVToolNix_GUI.AppImage /usr/local/bin/mkvpropedit
!sudo chmod a+rx /usr/local/bin/mkvpropedit
!sudo ln -s /usr/local/bin/MKVToolNix_GUI.AppImage /usr/local/bin/mkvmerge
!sudo chmod a+rx /usr/local/bin/mkvmerge
!sudo ln -s /usr/local/bin/MKVToolNix_GUI.AppImage /usr/local/bin/mkvextract
!sudo chmod a+rx /usr/local/bin/mkvextract
!sudo ln -s /usr/local/bin/MKVToolNix_GUI.AppImage /usr/local/bin/mkvinfo
!sudo chmod a+rx /usr/local/bin/mkvinfo

clear_output()
!mkvmerge --version

### __mkvmerge__

* You can __add__ tracks while running the cell.
* Don't use `#` and `"` in `mkvTitle`.
* Add XML file path for `globalTags`, `segmentInfo`.
* Choose a `splitMode` and add `splitArgument` __according__ to the `splitMode`.
* Chapters are accpeted __both__ `XML` and `OGM_txt` files.
* When adding a language use __laguage codes__.
* If you don't want to fill a field, leave it blank.
* If you don't know what is the relevant `mime-type`, also leave it blank in `mimeType`.
* When adding track `default`, `forced` flags, If you leave it blank, it's value will be __"No"__.
* For all `[y/n]` inputs __"y"__ for __"yes"__ and __"n"__ for __"no"__.
* `webmCompliantFile`: Create a WebM compliant file instead of mkv file output.
* While adding tracks if you have done adding, leave `inputFile` as blank to continue.
* When saving options are asked, enter folder path that you want to save your output file in `Enter folder path: ` and enter a file name to add to your output file in `Enter a name to save: `.
* If you leave both `Enter folder path: ` and `Enter a name to save: `, output folder will be `/content/` and your file will be renamed as the string that you gave in `mkvTitle`. If you haven't added `mkvTitle`, then your file will be renamed as same as the first inputed file's name. If a file exsists with the name generated for file, this will add `_new` to the end.

In [ ]:
#@markdown <h3><b>General</b></h3>
mkvTitle = "" #@param {type:"string"}
globalTags = "" #@param {type:"string"}
segmentInfo = "" #@param {type:"string"}
#@markdown <h3><b>Split</b></h3>
splitMode = "Do not split" #@param ["Do not split", "After output size", "After output duration", "After specific timestamps", "by parts based on timestamps", "by parts based on frame/field numbers", "After frame/field numbers", "Before chapters"]
splitArgument = "" #@param {type:"string"}
maxSplits = 0 #@param {type:"integer"}
linkFiles = False #@param {type:"boolean"}
#@markdown <h3><b>Chapters</b></h3>
chaptersFile = "" #@param {type:"string"}
chaptersLang = "" #@param {type:"string"}
#@markdown <h3><b>Attachments</b></h3>
attachment = "" #@param {type:"string"}
name = "" #@param {type:"string"}
mimeType = "" #@param {type:"string"}
description = "" #@param {type:"string"}
#@markdown <h3><b>Miscellaneous</b></h3>
webmCompliantFile = False #@param {type:"boolean"}
disableTrackStatisticsTags = False #@param {type:"boolean"}

import os
import json
import subprocess
from prettytable import PrettyTable

class mkvmerge(object):
    def __init__(self):
        self.__input_files = []
        self.__input_cmds = []
        self.__input_codes = ""

    def __get_json(self):
        json_cmd = [
            "mkvmerge",
            "--identify",
            "--identification-format",
            "json",
            os.path.abspath(self.input_file)
        ]
        json_data = subprocess.check_output(json_cmd, stderr=subprocess.DEVNULL)
        json_data = json.loads(json_data)

        return json_data
    
    def __get_info(self, i):
        json_data = self.__get_json()
        self.id = json_data["tracks"][i].get("id")
        self.language = json_data["tracks"][i]["properties"].get("language")
        self.codec = json_data["tracks"][i].get("codec")
        self.track_type = json_data["tracks"][i].get("type")

    def __view_tracks(self, length):
        table = PrettyTable(['id ', 'type', 'language', 'codec'])

        for i in range(length):
            self.__get_info(i)
            table.add_row([self.id, self.track_type, self.language, self.codec])
        
        table.align['id '] = "c"
        table.align['type'] = "l"
        table.align['language'] = "c"
        table.align['codec'] = "l"

        print(table)

    def get_start(self):
        while True:
            self.input_file = input("\ninputFile: ")
            if self.input_file == "":
                break
            self.__input_files.append(self.input_file)
            json_data = self.__get_json()

            track_count = len(json_data.get("tracks"))
            track_id = 0

            if track_count > 1:
                print()
                self.__view_tracks(track_count)
                track_id = int(input("\ntrack ID: "))
                
                if json_data["tracks"][track_id].get("type") == "video":
                    display_dimensions = json_data["tracks"][track_id]["properties"].get('display_dimensions')
                    self.__input_codes += f'--no-audio --no-subtitles --no-attachments --no-track-tags --no-global-tags --no-chapters --display-dimensions {track_id}:{display_dimensions} '
                
                elif json_data["tracks"][track_id].get("type") == "audio":
                    self.__input_codes += f'--no-video --no-subtitles --no-attachments --no-track-tags --no-global-tags --no-chapters '
                
                elif json_data["tracks"][track_id].get("type") == "subtitles":
                    self.__input_codes += f'--no-video --no-audio --no-attachments --no-track-tags --no-global-tags --no-chapters '

            track_title = input("|\n|--trackTitle: ")
            if track_title: self.__input_codes += f'--track-name {track_id}:"{track_title}" '

            track_language = input("|--trackLanguage: ")
            if track_language: self.__input_codes += f'--language {track_id}:{track_language} '

            track_default = input("|--Set track as default? [y/n] ")
            if track_default.lower() == "n" or track_default == "": self.__input_codes += f'--default-track-flag {track_id}:no '
            elif track_default.lower() == "y": self.__input_codes += f'--default-track-flag {track_id}:yes '

            track_forced = input("|--Set track as forced? [y/n] ")
            if track_forced.lower() == "n" or track_forced == "": self.__input_codes += f'--forced-display-flag {track_id}:no '
            elif track_forced.lower() == "y": self.__input_codes += f'--forced-display-flag {track_id}:yes '

            self.__input_codes += f'"{self.input_file}" '
            self.__input_cmds.append(self.__input_codes)
            self.__input_codes = ""
        
        if globalTags != "": self.__input_codes += f'--global-tags "{globalTags}" '
        if segmentInfo != "": self.__input_codes += f'--segmentinfo "{segmentInfo}" '

        if splitMode == "After output size": self.__input_codes += f'--split size:{splitArgument} '
        elif splitMode == "After output duration": self.__input_codes += f'--split duration:{splitArgument} '
        elif splitMode == "After specific timestamps": self.__input_codes += f'--split timestamps:{splitArgument} '
        elif splitMode == "by parts based on timestamps": self.__input_codes += f'--split parts:{splitArgument} '
        elif splitMode == "by parts based on frame/field numbers": self.__input_codes += f'--split parts-frames:{splitArgument} '
        elif splitMode == "After frame/field numbers": self.__input_codes += f'--split frames:{splitArgument} '
        elif splitMode == "Before chapters": self.__input_codes += f'--split chapters:{splitArgument} '
        
        if maxSplits > 1: self.__input_codes += f'--split-max-files {maxSplits} '
        if linkFiles: self.__input_codes += f'--link '

        if chaptersLang != "": self.__input_codes += f'--chapter-language {chaptersLang} '
        if chaptersFile != "": self.__input_codes += f'--chapters "{chaptersFile}" '

        if attachment != "":
            if name != "":
                attachment_ext = os.path.splitext(attachment)[1]
                name_ext = os.path.splitext(name)[1]
                if name_ext == attachment_ext: attachment_ext = ""
                self.__input_codes += f'--attachment-name "{name}{attachment_ext}" '
            if mimeType != "": self.__input_codes += f'--attachment-mime-type {mimeType} '
            if description != "": self.__input_codes += f'--attachment-description "{description}" '
            self.__input_codes += f'--attach-file "{attachment}" '

        print("\nSave output file to...\n|")
        output_dir = input("|--Enter folder path: ")
        output_name = input("|--Enter a name to save: ")
        print()

        if not output_dir: output_dir = "/content/"
        if not output_name:
            if not mkvTitle:
                if webmCompliantFile: output_name = os.path.splitext(os.path.basename(self.__input_files[0]))[0] + '.webm'
                else: output_name = os.path.splitext(os.path.basename(self.__input_files[0]))[0] + '.mkv'
            else:
                if webmCompliantFile: output_name = mkvTitle + '.webm'
                else: output_name = mkvTitle + '.mkv'
        else:
            if not (output_name.endswith('.webm') or output_name.endswith('.mkv')):
                if webmCompliantFile: output_name += '.webm'
                else: output_name += '.mkv'

        output_dir = os.path.join(os.path.abspath(output_dir), output_name)
        if os.path.exists(output_dir):
            output_dir = f"{os.path.splitext(output_dir)[0]}_new{os.path.splitext(output_dir)[1]}"
        command_line = f'--output "{output_dir}" '

        if webmCompliantFile: command_line += f'--webm '
        if disableTrackStatisticsTags: command_line += f'--disable-track-statistics-tags '

        command_line += ''.join(self.__input_cmds) + self.__input_codes
        if mkvTitle != "": command_line += f'--title "{mkvTitle}"'

        return command_line
    
mkv_merge = mkvmerge()
!mkvmerge {mkv_merge.get_start()}

### __mkvextract__

* You can extract __all tracks, attachments, chapters, tags, cues, cue sheet, timecodes__.
* Also you can extract a __single track__ by selecting `extractMode: Single Track` option.
* While extracting chapters, enter `chapters extract type?` as `xml` or `ogm` when asked for extract in that format

In [ ]:
inputFile = "" #@param {type:"string"}
extractMode = "Tracks" #@param ["Tracks", "Chapters", "Attachments", "Tags", "Cue Sheet", "Timecodes", "Cues", "Single Track"]

import os
import json
import subprocess
from prettytable import PrettyTable

class mkvextract(object):
    def __init__(self):
        self.input_file = os.path.abspath(inputFile)

    def __get_json(self):
        json_cmd = [
            "mkvmerge",
            "--identify",
            "--identification-format",
            "json",
            os.path.abspath(self.input_file)
        ]
        json_data = subprocess.check_output(json_cmd, stderr=subprocess.DEVNULL)
        json_data = json.loads(json_data)
        return json_data
    
    def __get_info(self, json_data, i:int):
        self.id = json_data["tracks"][i].get("id")
        self.language = json_data["tracks"][i]["properties"].get("language")
        self.language_ietf = json_data["tracks"][i]["properties"].get("language_ieft")
        self.codec = json_data["tracks"][i].get("codec")
        self.track_type = json_data["tracks"][i].get("type")
        self.track_name = json_data["tracks"][i]["properties"].get('track_name')
        self.codec_id = json_data["tracks"][i]["properties"].get('codec_id')

    def __view_tracks(self, length):
        table = PrettyTable(['id ', 'type', 'language', 'codec', 'name'])

        for i in range(length):
            json_data = self.__get_json()
            self.__get_info(json_data, i)
            table.add_row([self.id, self.track_type, self.language, self.codec, self.track_name])
        
        table.align['id '] = "c"
        table.align['type'] = "l"
        table.align['language'] = "c"
        table.align['codec'] = "l"
        table.align['name'] = "l"

        print(table)

    def __make_extract_dir(self):
        input_dir = os.path.dirname(os.path.abspath(self.input_file))
        input_name = os.path.splitext(os.path.basename(self.input_file))[0]
        os.makedirs(os.path.join(input_dir, input_name), exist_ok=True)
        return os.path.join(input_dir, input_name)

    def __process_file(self, i:int):
        json_data = self.__get_json()
        if self.track_type == "video":
            pixel_dimensions = json_data["tracks"][i]["properties"].get('pixel_dimensions')
            self.extract_name = f'Track_{self.id}_[{self.track_type}]_[{pixel_dimensions}]_[{self.language}]'
        elif self.track_type == "audio":
            audio_channels = json_data["tracks"][i]["properties"].get('audio_channels')
            audio_sampling_frequency = json_data["tracks"][i]["properties"].get('audio_sampling_frequency')
            self.extract_name = f'Track_{self.id}_[{self.track_type}]_[{audio_channels}CH]_[{audio_sampling_frequency / 1000}kHz]_[{self.language}]'
        elif self.track_type == "subtitles":
            self.extract_name = f'Track_{self.id}_[{self.track_type}]_[{self.language}]'

        if "AVC" in self.codec_id:
            self.extract_name += ".h264"
        elif "HEVC" in self.codec_id:
            self.extract_name += ".h265"
        elif "V_VP8" in self.codec_id or "V_VP9" in self.codec_id:
            self.extract_name += ".ivf"
        elif "V_AV1" in self.codec_id:
            self.extract_name += ".ivf"
        elif "V_MPEG1" in self.codec_id or "V_MPEG2" in self.codec_id:
            self.extract_name += ".mpg"
        elif "V_REAL" in self.codec_id:
            self.extract_name += ".rm"
        elif "V_THEORA" in self.codec_id:
            self.extract_name += ".ogg"
        elif "V_MS/VFW/FOURCC" in self.codec_id:
            self.extract_name += ".avi"
        elif "AAC" in self.codec_id:
            self.extract_name += ".aac"
        elif "A_AC3" in self.codec_id:
            self.extract_name += ".ac3"
        elif "A_EAC3" in self.codec_id:
            self.extract_name += ".eac3"
        elif "ALAC" in self.codec_id:
            self.extract_name += ".caf"
        elif "DTS" in self.codec_id:
            self.extract_name += ".dts"
        elif "FLAC" in self.codec_id:
            self.extract_name += ".flac"
        elif "MPEG/L2" in self.codec_id:
            self.extract_name += ".mp2"
        elif "MPEG/L3" in self.codec_id:
            self.extract_name += ".mp3"
        elif "OPUS" in self.codec_id:
            self.extract_name += ".ogg"
        elif "PCM" in self.codec_id:
            self.extract_name += ".wav"
        elif "REAL" in self.codec_id:
            self.extract_name += ".ra"
        elif "TRUEHD" in self.codec_id:
            self.extract_name += ".thd"
        elif "MLP" in self.codec_id:
            self.extract_name += ".mlp"
        elif "TTA1" in self.codec_id:
            self.extract_name += ".tta"
        elif "VORBIS" in self.codec_id:
            self.extract_name += ".ogg"
        elif "WAVPACK4" in self.codec_id:
            self.extract_name += ".wv"
        elif "PGS" in self.codec_id:
            self.extract_name += ".sup"
        elif "ASS" in self.codec_id:
            self.extract_name += ".ass"
        elif "SSA" in self.codec_id:
            self.extract_name += ".ssa"
        elif "UTF8" in self.codec_id or "ASCII" in self.codec_id:
            self.extract_name += ".srt"
        elif "VOBSUB" in self.codec_id:
            self.extract_name += ".sub"
        elif "S_KATE" in self.codec_id:
            self.extract_name += ".ogg"
        elif "USF" in self.codec_id:
            self.extract_name += ".usf"
        elif "WEBVTT" in self.codec_id:
            self.extract_name += ".vtt"

    def __extract_tracks(self):
        extract_dir = self.__make_extract_dir()
        json_data = self.__get_json()

        extract_codes = []

        for i in range(len(json_data["tracks"])):
            self.__get_info(json_data, int(i))
            self.__process_file(int(i))
            extract_codes.append(f'{self.id}:"{os.path.join(extract_dir, self.extract_name)}"')

        cmd = f'mkvextract "{self.input_file}" tracks {" ".join(extract_codes)}'
        print("Extracting...")
        !{cmd}

    def __extract_a_single_track(self):
        extract_dir = self.__make_extract_dir()
        json_data = self.__get_json()

        self.__view_tracks(len(json_data["tracks"]))
        i = input("\ntrack ID: ")

        self.__get_info(json_data, int(i))
        self.__process_file(int(i))

        cmd = f'mkvextract "{self.input_file}" tracks {self.id}:"{os.path.join(extract_dir, self.extract_name)}"'
        print("\nExtracting...")
        !{cmd}

    def __extract_attachments(self):
        extract_dir = self.__make_extract_dir()
        json_data = self.__get_json()

        if len(json_data.get("attachments")) > 0:
            for attachment in json_data.get("attachments"):
                attachment_id = attachment.get("id")
                attachment_name = attachment.get("file_name")
                extractParam = (f'')
                cmd = f'mkvextract "{self.input_file}" attachments {attachment_id}:"{os.path.join(extract_dir, attachment_name)}"'
                !{cmd}
        elif len(json_data.get("attachments")) == 0:
            print("No attachments are available.")

    def __extract_chapters(self, chapter_type):
        extract_dir = self.__make_extract_dir()
        json_data = self.__get_json()

        if len(json_data.get("chapters")) > 0:
            if chapter_type == "xml": cmd = f'mkvextract "{self.input_file}" chapters "{os.path.join(extract_dir, "Chapters.xml")}"'
            elif chapter_type == "ogm": cmd = f'mkvextract "{self.input_file}" chapters --simple "{os.path.join(extract_dir, "Chapters.txt")}"'
            !{cmd}
            print("Extracted.")
        elif len(json_data.get("chapters")) == 0:
            print("No chapters are available.")

    def __extract_tags(self):
        extract_dir = self.__make_extract_dir()
        json_data = self.__get_json()

        if len(json_data.get("global_tags")) or len(json_data.get("track_tags")) > 0:
            cmd = f'mkvextract "{self.input_file}" tags "{os.path.join(extract_dir, "Tags.xml")}"'
            !{cmd}
            print("Extracted.")
        elif (len(json_data.get("global_tags")) == 0) and (len(json_data.get("track_tags")) == 0):
            print("No tags are available.")

    def __extract_timecodes(self):
        extract_dir = self.__make_extract_dir()
        json_data = self.__get_json()

        extract_codes = []

        for i in range(len(json_data["tracks"])):
            self.__get_info(json_data, int(i))
            extract_name = f'Track_{self.id}_[{self.track_type}]_tc.txt'
            extract_codes.append(f'{self.id}:"{os.path.join(extract_dir, extract_name)}"')

        cmd = f'mkvextract "{self.input_file}" timecodes_v2 {" ".join(extract_codes)}'
        print("Extracting...")
        !{cmd}

    def __extract_cues(self):
        extract_dir = self.__make_extract_dir()
        json_data = self.__get_json()

        extract_codes = []

        for i in range(len(json_data["tracks"])):
            self.__get_info(json_data, int(i))
            extract_name = f'Track_{self.id}_[{self.track_type}]_cues.txt'
            extract_codes.append(f'{self.id}:"{os.path.join(extract_dir, extract_name)}"')

        cmd = f'mkvextract "{self.input_file}" cues {" ".join(extract_codes)}'
        print("Extracting...")
        !{cmd}

    def __extract_cuesheet(self):
        extract_dir = self.__make_extract_dir()
        cmd = f'mkvextract "{self.input_file}" cuesheet "{os.path.join(extract_dir, "CueSheet.cue")}"'
        !{cmd}
        print("Extracted.")

    def get_start(self):
        if extractMode == "Tracks": self.__extract_tracks()
        elif extractMode == "Single Track": self.__extract_a_single_track()
        elif extractMode == "Chapters":
            chapter_type = input("chapters extract type? [xml/ogm] ")
            if chapter_type.lower() == "xml" or chapter_type.lower() == "ogm": self.__extract_chapters(chapter_type.lower())
            else: print("Incorrect input.")
        elif extractMode == "Attachments": self.__extract_attachments()
        elif extractMode == "Tags": self.__extract_tags()
        elif extractMode == "Timecodes": self.__extract_timecodes()
        elif extractMode == "Cues": self.__extract_cues()
        elif extractMode == "Cue Sheet": self.__extract_cuesheet()

mkv_extract = mkvextract()
mkv_extract.get_start()

### __mkvpropedit__

* You can edit __segment info, track info, chapters, attachments, tags__ in a mkv file.
* If you want to __delete statistic tags__ from tracks select `deleteTrackStatisticsTags`

In [ ]:
mkvFile = "" #@param {type:"string"}
handleMode = "None" #@param ["None", "Segment Info", "Tracks", "Chapters", "Attachments", "Tags"]
deleteTrackStatisticsTags = False #@param {type:"boolean"}

import os
import math
import json
import subprocess
from prettytable import PrettyTable

class mkvpropedit():
    def __init__(self):
        self.input_file = os.path.abspath(mkvFile)
        self.__input_codes = ""

    def __get_json(self):
        json_cmd = [
            "mkvmerge",
            "--identify",
            "--identification-format",
            "json",
            os.path.abspath(self.input_file)
        ]
        json_data = subprocess.check_output(json_cmd, stderr=subprocess.DEVNULL)
        json_data = json.loads(json_data)
        return json_data
    
    def __get_track_info(self, json_data, i:int):
        self.id = json_data["tracks"][i].get("id")
        self.language = json_data["tracks"][i]["properties"].get("language")
        self.language_ietf = json_data["tracks"][i]["properties"].get("language_ieft")
        self.codec = json_data["tracks"][i].get("codec")
        self.track_type = json_data["tracks"][i].get("type")
        self.track_name = json_data["tracks"][i]["properties"].get('track_name')
        self.codec_id = json_data["tracks"][i]["properties"].get('codec_id')

    def __get_attachment_info(self, json_data, i:int):
        self.attachment_id = json_data["attachments"][i].get("id")
        self.attachment_name = json_data["attachments"][i].get("file_name")
        self.attachment_type = json_data["attachments"][i].get("content_type")
        self.attachment_description = json_data["attachments"][i].get("description")
        if self.attachment_description == "": self.attachment_description = "None"
        self.attachment_uid = json_data["attachments"][i]["properties"].get("uid")
        self.attachment_size = self.__convert_size(json_data["attachments"][i].get("size"))

    def __convert_size(self, size_bytes):
        if size_bytes == 0:
            return "0B"
        size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
        i = int(math.floor(math.log(size_bytes, 1024)))
        p = math.pow(1024, i)
        s = round(size_bytes / p, 2)
        return f"{s} {size_name[i]}"

    def __view_tracks(self, json_data, length):
        table = PrettyTable(['id ', 'type', 'language', 'codec', 'name'])

        for i in range(length):
            self.__get_track_info(json_data, i)
            table.add_row([self.id, self.track_type, self.language, self.codec, self.track_name])
        
        table.align['id '] = "c"
        table.align['type'] = "l"
        table.align['language'] = "c"
        table.align['codec'] = "l"
        table.align['name'] = "l"
        print(table)
    
    def __view_attachments(self, json_data, length):
        table = PrettyTable(['id ', 'name', 'mime-type', 'size', 'uid', 'description'])

        for i in range(length):
            self.__get_attachment_info(json_data, i)
            table.add_row([self.attachment_id, self.attachment_name, self.attachment_type, self.attachment_size, self.attachment_uid, self.attachment_description])
        
        table.align['id '] = "c"
        table.align['name'] = "l"
        table.align['mime-type'] = "l"
        table.align['size'] = "l"
        table.align['uid'] = "l"
        table.align['description'] = "l"
        print(table)

    def __edit_tracks(self):
        json_data = self.__get_json()
        self.__view_tracks(json_data, len(json_data.get("tracks")))
        track_id = int(input('\ntrack ID: '))
        mkvpropedit_id = track_id + 1
        property_type = input("\nproperty type to edit?\n\n1: Track Title\n2: Track Language\n3: Track Default\n4: Track Forced\n\n[1/2/3/4] ")
        if property_type in ["1", "2", "3", "4"]:
            if property_type == "1":
                edit_type = input("\nHow to edit?\n\n1: Add\n2: Change\n3: Delete\n\n[1/2/3] ")
                if edit_type in ["1", "2"]:
                    track_title = input("\nTitle: ")
                    if edit_type == "1": self.__input_codes += f'--add name="{track_title}"'
                    elif edit_type == "2": self.__input_codes += f'--set name="{track_title}"'
                elif edit_type == "3": self.__input_codes += '--delete name'
                else: return "Incorrect input"
            elif property_type == "2":
                edit_type = input("\nHow to edit?\n\n1: Add\n2: Change\n3: Delete\n\n[1/2/3] ")
                if edit_type in ["1", "2"]:
                    track_lang = input("\nLanguage: ")
                    if edit_type == "1": self.__input_codes += f'--add language={track_lang}'
                    elif edit_type == "2": self.__input_codes += f'--set language={track_lang}'
                elif edit_type == "3": self.__input_codes += '--delete language'
                else: return "Incorrect input"
            elif property_type == "3":
                track_default = input(f'\nSet this track as default?\n\n0: No\n1: Yes\n\n[0/1] ')
                if track_default in ["0", "1"]: self.__input_codes += f'--set flag-default={track_default}'
                else: return "Incorrect input"
            elif property_type == "4":
                track_forced = input(f'\nSet this track as forced?\n\n0: No\n1: Yes\n\n[0/1] ')
                if track_forced in ["0", "1"]: self.__input_codes += f'--set flag-forced={track_forced}'
                else: return "Incorrect input"
            return f'"{self.input_file}" --edit track:{mkvpropedit_id} {self.__input_codes} '
        else: return "Incorrect input"

    def __edit_segment_info(self):
        edit_type = input("Segment infomation : Title\n\n1: Add\n2: Change\n3: Delete\n\n[1/2/3] ")
        if edit_type in ["1", "2"]:
            mkv_title = input("\nTitle: ")
            if edit_type == "1": self.__input_codes += f'--add title="{mkv_title}"'
            elif edit_type == "2": self.__input_codes += f'--set title="{mkv_title}"'
        elif edit_type == "3": self.__input_codes += '--delete title'
        else: return "Incorrect input"
        return f'"{self.input_file}" --edit info {self.__input_codes} '
    
    def __edit_chapters(self):
        edit_type = input("Chapters\n\n1: Add\n2: Remove\n\n[1/2] ")
        if edit_type == "1":
            file_path = input("\nChapter file path: ")
            self.__input_codes += f'--chapters "{file_path}"'
        elif edit_type == "2": self.__input_codes += '--chapters ""'
        else: return "Incorrect input"
        return f'"{self.input_file}" --edit info {self.__input_codes} '
    
    def __edit_attachments(self):
        edit_type = input(f'Attachments\n\n1: Add\n2: Replace\n3: Update\n4: Remove\n5: List\n\n[1/2/3/4/5] ')
        if edit_type == "1":
            attachment_path = input(f'\nattachment path: ')
            attachment_ext = os.path.splitext(os.path.abspath(attachment_path))[-1]
            attachment_name = input(f'\nName: ')
            attachment_mimetype = input(f'MIME-Type: ')
            attachment_desc = input(f'Description: ')
            attachment_uid = input(f'UID: ')

            if attachment_name: self.__input_codes += f'--attachment-name "{attachment_name}{attachment_ext}" '
            if attachment_mimetype: self.__input_codes += f'--attachment-mime-type "{attachment_mimetype}" '
            if attachment_desc: self.__input_codes += f'--attachment-description "{attachment_desc}" '
            if attachment_uid: self.__input_codes += f'--attachment-uid "{attachment_uid}" '

            self.__input_codes += f'--add-attachment "{attachment_path}" '
            return f'"{self.input_file}" {self.__input_codes} '
        elif edit_type in ["2", "3", "4", "5"]:
            print()
            json_data = self.__get_json()
            self.__view_attachments(json_data, len(json_data.get("attachments")))

            if edit_type in ["2", "3"]:
                attachment_id = input("\nattachment ID: ")
                attachment_path = input(f'\nattachment path: ')
                attachment_ext = os.path.splitext(os.path.abspath(attachment_path))[-1]
                attachment_name = input(f'\nName: ')
                attachment_mimetype = input(f'MIME-Type: ')
                attachment_desc = input(f'Description: ')
                attachment_uid = input(f'UID: ')

                if attachment_name: self.__input_codes += f'--attachment-name "{attachment_name}{attachment_ext}" '
                if attachment_mimetype: self.__input_codes += f'--attachment-mime-type "{attachment_mimetype}" '
                if attachment_desc: self.__input_codes += f'--attachment-description "{attachment_desc}" '
                if attachment_uid: self.__input_codes += f'--attachment-uid "{attachment_uid}" '

                if edit_type == "2": self.__input_codes += f'--replace-attachment {attachment_id}:"{attachment_path}" '
                elif edit_type == "3": self.__input_codes += f'--update-attachment {attachment_id} '

                return f'"{self.input_file}" {self.__input_codes} '
            elif edit_type == "4":
                attachment_id = input("\nattachment ID: ")
                self.__input_codes += f'--delete-attachment {attachment_id} '
                return f'"{self.input_file}" {self.__input_codes} '
            return "list"
        else: return "Incorrect input"

    def __edit_tags(self):
        edit_type = input('Tags\n\n1: All tags\n2: Global tags\n3: Track tags\n\n[1/2/3] ')
        if edit_type in ["1", "2", "3"]:
            if edit_type == "1":
                tag_type = input('\nAll Tags\n\n1: Add\n2: Remove\n\n[1/2] ')
                if tag_type in ["1", "2"]:
                    if tag_type == "1":
                        xml_path = input(f'\nXML file path: ')
                        self.__input_codes += f'--tags all:"{xml_path}" '
                    elif tag_type == "2": self.__input_codes += '--tags all: '
                    return f'"{self.input_file}" {self.__input_codes} '
                else: return "Incorrect input"
            elif edit_type == "2":
                tag_type = input('\nGlobal Tags\n\n1: Add\n2: Remove\n\n[1/2] ')
                if tag_type in ["1", "2"]:
                    if tag_type == "1":
                        xml_path = input(f'\nXML file path: ')
                        self.__input_codes += f'--tags global:"{xml_path}" '
                    elif tag_type == "2": self.__input_codes += '--tags global: '
                    return f'"{self.input_file}" {self.__input_codes} '
                else: return "Incorrect input"
            elif edit_type == "3":
                print()
                json_data = self.__get_json()
                self.__view_tracks(json_data, len(json_data.get("tracks")))
                track_id = input(f'\ntrack ID: ')
                mkvpropedit_id = int(track_id) + 1
                tag_type = input('\nTrack Tags\n\n1: Add\n2: Remove\n\n[1/2] ')
                if tag_type in ["1", "2"]:
                    if tag_type == '1':
                        xml_path = input(f'\nXML file path: ')
                        self.__input_codes += f'--tags track:{mkvpropedit_id}:"{xml_path}" '
                    elif tag_type == '2': self.__input_codes += f'--tags track:{mkvpropedit_id}: '
                    return f'"{self.input_file}" {self.__input_codes} '
                else: return "Incorrect input"
        else: return "Incorrect input"

    def get_start(self):
        if deleteTrackStatisticsTags:
            if handleMode == 'None': return f'{self.input_file} --delete-track-statistics-tags'
            elif handleMode == 'Tracks': return f'{self.__edit_tracks()} --delete-track-statistics-tags'
            elif handleMode == 'Segment Info': return f'{self.__edit_segment_info()} --delete-track-statistics-tags'
            elif handleMode == 'Chapters': return f'{self.__edit_chapters()} --delete-track-statistics-tags'
            elif handleMode == 'Attachments': return f'{self.__edit_attachments()} --delete-track-statistics-tags'
            elif handleMode == 'Tags': return f'{self.__edit_tags()} --delete-track-statistics-tags'
        else:
            if handleMode == 'None': return f'{self.input_file}'
            elif handleMode == 'Tracks': return self.__edit_tracks()
            elif handleMode == 'Segment Info': return self.__edit_segment_info()
            elif handleMode == 'Chapters': return self.__edit_chapters()
            elif handleMode == 'Attachments': return self.__edit_attachments()
            elif handleMode == 'Tags': return self.__edit_tags()

mkv_propedit = mkvpropedit()
cmd = mkv_propedit.get_start()
print()
if not "Incorrect input" in cmd and not "list" in cmd:
    !mkvpropedit {cmd}
elif "Incorrect input" in cmd:
    print("Incorrect input")

### __Matroska/WebM Tags__

* Enter __mkv or webm file's path__ in `mkvFile`.
* If you want to delete `Title`, leave it as blank.
* You can add __tags__ using a __text document__, but the text document's content must be as formatted as below.<br>
`Tag name: Tag value, Tag value`
* This doesn't add tags as __multiple tags__, adds all values as a single tag.
* Use __official tag names__ to add __matroska official tags__. see [here](https://www.matroska.org/technical/tagging.html)

In [ ]:
mkvFile = "" #@param {type:"string"}
saveXML = True #@param {type:"boolean"}

import os
from xml.etree.ElementTree import Element, tostring

class mkvtagger(object):
    def __init__(self):
        self.tags_dict = {}
        self.input_file = os.path.abspath(mkvFile)
        self.__get_xml()

    def __get_xml(self):
        source_dir = os.path.dirname(self.input_file)
        source_name = os.path.splitext(os.path.basename(self.input_file))[0]
        self.xml_path = os.path.join(source_dir, f'{source_name}_[TagsXML].xml')

    def __get_tags(self):
        print("Matroska/WebM Tags")
        print("│")
        self.mkv_title = input("├── Title: ")
        print("│")
        tags_txt = input("├── Do you want to add tags from a text document? [y/n] ")
        if tags_txt.lower() in ["y", "yes"]:
            txt_path = input("├── Text document's path: ")
            print("│")
            with open(os.path.abspath(txt_path), 'r') as txt:
                for custom_tag in txt:
                    custom_tag = custom_tag.strip()
                    tag_name = custom_tag.split(":")[0]
                    tag_name = tag_name.strip()
                    tag_values = custom_tag.split(":")[1]
                    tag_values = tag_values.split(",")

                    i = 0
                    for value in tag_values:
                        value = value.strip()
                        tag_values[i] = value
                        i += 1
                    
                    tag_value = ", ".join(tag_values)
                    self.tags_dict[tag_name] = tag_value
                    print(f"├── {tag_name}: {tag_value}")
        elif tags_txt.lower() in ["n", "no"]:
            print("│")
            print("├── Type [tag name] first and [tag value] second.")
            print("├── When finished, leave 'Tag name: ' as blank.")
            print("│")

            while True:
                tag_name = input("├── Tag name: ")
                if tag_name == "":
                    break
                else:
                    tag_value = input(f'├── {tag_name}: ')
                    self.tags_dict[tag_name] = tag_value
        else:
            print('│\n└── Input is invalid. Try again...')
            print()
            return "error"

    def __create_xml(self):
        node_tags = Element('Tags')
        node_tag = Element('Tag')
        for name, value in self.tags_dict.items():
            node_simple = Element('Simple')
            node_name = Element('Name')
            node_name.text = name
            node_simple.append(node_name)
            node_string = Element('String')
            node_string.text = value
            node_simple.append(node_string)
            node_tag.append(node_simple)
        node_tags.append(node_tag)
        xml_data = tostring(node_tags)

        with open(self.xml_path, 'wb') as xml:
            xml.write(xml_data)

    def get_start(self):
        t = self.__get_tags()
        self.__create_xml()

        if t != "error":
            command_line = f'"{self.input_file}" --tags all:"{self.xml_path}" --edit info '

            if not self.mkv_title: command_line += "--delete title "
            else: command_line += f'--set title="{self.mkv_title}" '
            
            print("│")
            encoded_date = input('├── Do you want to remove encoded date? [y/n] ')
            writing_application = input('├── Do you want to remove writing application? [y/n] ')
            writing_library = input('└── Do you want to remove writing library? [y/n] ')
            print()

            if encoded_date in ["y", "yes"]: command_line += "--delete date "
            if writing_application in ["y", "yes"]: command_line += '--set writing-application="" '
            if writing_library in ["y", "yes"]: command_line += '--set muxing-application="" '

            if not saveXML: os.remove(self.xml_path)
            return command_line
    
mkv_tagger = mkvtagger()
print()
!mkvpropedit {mkv_tagger.get_start()}